In [45]:
import psycopg2
import pandas as pd
import seaborn as sns
import configparser
import numpy as np
import h5py

In [2]:
# Read database configuration from the .ini file
config = configparser.ConfigParser()
config.read("../database.ini")

db_params = {
    "dbname": config["postgresql"]["dbname"],
    "user": config["postgresql"]["user"],
    "password": config["postgresql"]["password"],
    "host": config["postgresql"]["host"],
}


In [3]:
with psycopg2.connect(**db_params) as conn:
    # Parameterized query
    query = """
        select id
    ,title
    ,score
    ,by as author
    from hacker_news.items 

    where type = 'story'
    and title is not null
    LIMIT 100000
    """


In [4]:
# Read directly into DataFrame 
df = pd.read_sql_query(query, conn)
    
# Optionally specify column dtypes
df = pd.read_sql_query(
        query,
        conn
        # ,dtype={
        #     'integer_column': 'int32',
        #     'float_column': 'float64'
        # },
        # parse_dates=['date_column']  # Automatically parse date columns
    )

df.head()

/var/folders/40/wsz96msx0yj64gpqth1rs3b40000gn/T/ipykernel_53539/2978195756.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)
/var/folders/40/wsz96msx0yj64gpqth1rs3b40000gn/T/ipykernel_53539/2978195756.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(


,id,title,score,author
0,3150000,"""What May Happen in the Next Hundred Years"", f...",19,ColinWright
1,3500001,Getting Started with JavaScript Unit Testing,1,hncj
2,3150001,"Armstrong, the Django-based and open-source ne...",2,andymboyle
3,6050000,Why Web Reviewers Make Up Bad Things,1,digisth
4,150000,You Weren't Meant to Have a Boss: The Cliff Notes,1,jazzdev


In [5]:
# Count words in each title
df['count'] = df['title'].str.split().str.len()

# Remove special characters and digits using str.replace()
df['title'] = df['title'].apply(lambda x: x.lower())  # Convert to lowercase
df['title'] = df['title'].str.replace(r'[^\w\s]', '', regex=True)  # Remove special chars
df['title'] = df['title'].str.replace(r'\d+', '', regex=True)  # Remove digits

# Split titles into words and create dictionary
df['words'] = df['title'].str.split()

# Export DataFrame to CSV
df.to_csv('hn_titles_sample.csv', index=False)

df.head()

,id,title,score,author,count,words
0,3150000,what may happen in the next hundred years from c,19,ColinWright,10,"[what, may, happen, in, the, next, hundred, ye..."
1,3500001,getting started with javascript unit testing,1,hncj,6,"[getting, started, with, javascript, unit, tes..."
2,3150001,armstrong the djangobased and opensource news ...,2,andymboyle,10,"[armstrong, the, djangobased, and, opensource,..."
3,6050000,why web reviewers make up bad things,1,digisth,7,"[why, web, reviewers, make, up, bad, things]"
4,150000,you werent meant to have a boss the cliff notes,1,jazzdev,10,"[you, werent, meant, to, have, a, boss, the, c..."


In [24]:

from gensim.models import Word2Vec

model = Word2Vec.load("word2vec_model.model")

def get_vector(x):
    try:
        return np.array(model.wv[x])
    except KeyError:
        return None

df['word_vectors'] = df['words'].apply(lambda word_list: [get_vector(word) for word in word_list])

# Remove None values from word vectors
df['word_vectors'] = df['word_vectors'].apply(lambda x: [v for v in x if v is not None])

#Covert to numpy array
df['word_vectors'] = df['word_vectors'].apply(lambda x: np.array(x))

# Calculate mean vector for each title by taking mean across word vectors
df['mean_vector'] = df['word_vectors'].apply(lambda x: np.mean(x, axis=0))


df.head()

/Users/andreas.paxinos/Documents/github/mli/.venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/andreas.paxinos/Documents/github/mli/.venv/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


,id,title,score,author,count,words,word_vectors,mean_vector
0,3150000,what may happen in the next hundred years from c,19,ColinWright,10,"[what, may, happen, in, the, next, hundred, ye...","[[0.0038050802, 0.00086296524, 0.0029399444, 0...","[0.00077195687, 0.00042079733, 0.00012249911, ..."
1,3500001,getting started with javascript unit testing,1,hncj,6,"[getting, started, with, javascript, unit, tes...","[[-0.0035109387, -0.0021732112, 0.00224942, -0...","[-0.0017264247, -0.00029120408, 0.0007486603, ..."
2,3150001,armstrong the djangobased and opensource news ...,2,andymboyle,10,"[armstrong, the, djangobased, and, opensource,...","[[-0.0030918512, 0.00022298544, 0.0017556854, ...","[0.0006859072, 1.0250059e-05, 0.0010865574, -0..."
3,6050000,why web reviewers make up bad things,1,digisth,7,"[why, web, reviewers, make, up, bad, things]","[[0.0026316757, -0.0016147939, 0.0027679498, 0...","[-0.00013478259, -0.0004138446, -0.0004691301,..."
4,150000,you werent meant to have a boss the cliff notes,1,jazzdev,10,"[you, werent, meant, to, have, a, boss, the, c...","[[0.0036288062, -0.0037281888, 0.0011980163, 0...","[0.00026220054, -0.0013449227, 0.0005244772, -..."


In [30]:
#Check shape of word vectors column at index 0
print(df['word_vectors'][0].shape)

#Check shape of mean vector column at index 0
print(df['mean_vector'][0].shape)


(5, 256)
(256,)


In [54]:
# Drop unnecessary columns
df_clean = df.drop(['title', 'author', 'count', 'words', 'word_vectors'], axis=1)

# Remove rows where mean_vector is NaN
df_clean = df_clean.dropna(subset=['mean_vector'])


# Export cleaned DataFrame to CSV
df_clean.to_hdf('hn_title_mean_vectors_scores.h5', key='df_clean', mode='w')


/var/folders/40/wsz96msx0yj64gpqth1rs3b40000gn/T/ipykernel_53539/1292129051.py:9: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['mean_vector'], dtype='object')]

  df_clean.to_hdf('hn_title_mean_vectors_scores.h5', key='df_clean', mode='w')


In [49]:
df_clean.head()


Rows with NaN mean vectors:


,id,score,mean_vector


In [55]:
import os

file_size_bytes = os.path.getsize('hn_title_mean_vectors_scores.h5')
file_size_mb = file_size_bytes / (1024 * 1024)
print(f"File size: {file_size_mb:.2f} MB")


File size: 102.61 MB
